In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='e285d04e-8405-4384-a49d-4dda639b7191', project_access_token='p-47aff461317d8707510d69f1fbbd35a48d608ccb')
pc = project.project_context

![ibm cloud logo](./images/ibm-cloud.png)

# Case Study - Data ingestion

The goal of this case study is to put into practice the important concepts from module 1.  We will go through the basic process that begins with refining the business opportunity and ensuring that it is articulated using a scientific thought process.

The business opportunity and case study was first mentioned in Unit 2 of module 1 and like the AAVIAL company itself these data were created for learning purposes.  We will be using the AAVAIL example as a basis for this case study.  [Watch the video again](https://vimeo.com/348708995) if you need a refresher.  You will be gathering data from several provided sources, staging it for quality assurance and saving it in a target destination that is most appropriate.

Watch the video to review the important concepts from the units you just covered and to see an overview of the objectives for this case study.

In [2]:
from IPython.display import IFrame
IFrame('https://player.vimeo.com/video/354996550', width=600,height=400)

## Case study overall objectives

1. Gather all relevant data from the sources of provided data
2. Implement several checks for quality assurance 
3. Take the initial steps towards automation of the ingestion pipeline

## Getting started

Download this notebook and open it locally using a Jupyter server. Alternatively you may use Watson Studio.  To make using Watson Studio easier we have provided a zip archive file containing the files needed to complete this case study in Watson Studio. See the [Getting started with Watson Studio](m1-u5-5-watson-studio.rst) page.

**You will need the following files to complete this case study**

* [m1-u6-case-study.ipynb](m1-u6-case-study.ipynb)
* [m1-u6-case-study-solution.ipynb](./notebooks/m1-u6-case-study-solution.ipynb)
* [aavail-customers.db](./data/aavail-customers.db)
* [aavail-steams.csv](./data/aavail-streams.csv)

1. Fill in all of the places in this notebook marked with ***YOUR CODE HERE*** or ***YOUR ANSWER HERE***
2. When you have finished the case study there will be a short quiz

You may review the rest of this content as part of the notebook, but once you are ready to get started be ensure that you are working with a *live* version either as part of Watson Studio or locally.

## Data Sources

The data you will be sourcing from is contained in two sources.

1. A database ([SQLite](https://www.sqlite.org/index.html)) of `customer` data
2. A [CSV file](https://en.wikipedia.org/wiki/Comma-separated_values) of `stream` level data

   >You will create a simple data pipeline that
   (1) simplifies the data for future analysis
   (2) performs quality assurance checks.

The process of building *the data ingestion pipeline* entails extracting data, transforming it, and loading it into an appropriate data storage technology.  When constructing a pipeline it is important to keep in mind that they generally process data in batches.  For example, data may be compiled during the day and the batch could be processed during the night.  The data pipeline may also be optimized to execute as a streaming computation (i.e., every event is handled as it occurs).

## PART 1: Gathering the data

The following is an [Entity Relationship Diagram (ERD)](https://en.wikipedia.org/wiki/Entity%E2%80%93relationship_model) that details the tables and contents of the database.

<img src="./images/aavail-schema.svg" alt="customer database schema" style="width: 600px;"/>

In [3]:
## all the imports you will need for this case study
import os
import pandas as pd
import numpy as np
import sqlite3

Much of the data exist in a database.  You can connect to is using the `sqlite3` Python package with the function shown below.  Note that is is good practice to wrap your connect functions in a [try-except statement](https://docs.python.org/3/tutorial/errors.html) to cleanly handle exceptions.

In [4]:
DATA_DIR = pc.home + '/data'

In [5]:
import os

def copy_data_to_files(p):
    data_dir = p.project_context.home + '/data'

    if not os.path.exists( data_dir ):
        os.makedirs( data_dir )
    
    for fn in [data_asset['name'] for data_asset in p.get_files()]:
        with open( data_dir + '/' + fn, 'wb') as w:
            w.write(p.get_file(fn).read())
            
copy_data_to_files(project)

In [6]:
print(os.getcwd())
print(os.listdir())
print(os.listdir("/home/dsxuser/data"))

/home/dsxuser/work
[]
['aavail-customers.db', 'aavail-streams.csv']


In [7]:
def connect_db(file_path):
    try:
        conn = sqlite3.connect(file_path)
        print("...successfully connected to db\n")
    except Error as e:
        print("...unsuccessful connection\n",e)
    
    return(conn)

In [8]:
## make the connection to the database
conn = connect_db(DATA_DIR + '/aavail-customers.db')

## print the table names
tables = [t[0] for t in conn.execute("SELECT name FROM sqlite_master WHERE type='table';")]
print(tables)

...successfully connected to db

['CUSTOMERS', 'INVOICES', 'INVOICE_ITEMS']


In [9]:
df_invoice_items = pd.read_sql_query("SELECT invoice_item_id, invoice_item FROM INVOICE_ITEMS;", conn)
customer_types = df_invoice_items.set_index("invoice_item_id").T.to_dict("index")['invoice_item']

In [10]:
df_invoices = pd.read_sql_query("SELECT * FROM INVOICES;", conn)
df_invoices.head()

# df_invoices[["customer_id","invoice_item_id"]].invoice_item_id.replace(customer_types)

,generated_id,invoice_item_id,customer_id,signup_date,last_stream
0,1,3,1,2018-11-29,2019-03-18
1,2,3,2,2019-03-28,2019-06-08
2,3,2,3,2018-12-20,2019-05-31
3,4,3,4,2018-01-04,2018-03-29
4,5,1,5,2019-03-08,2019-05-24


### QUESTION 1:

**extract the relevant data from the DB**

Query the database and extract the following data into a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).
 
* Customer ID (integer)
* Last name
* First name
* DOB
* City
* State
* Country (the name NOT the country_id)
* Gender

Remember that that SQL is case-insensitive, but it is traditional to use ALL CAPS for SQL keywords. It is also a convention to end SQL statements with a semi-colon.  

#### Resources

* [W3 schools SQL tutorial](https://www.w3schools.com/sql)
* [W3 schools SQL joins](https://www.w3schools.com/sql/sql_join.asp)

In [11]:
## YOUR CODE HERE

df_customers = pd.read_sql_query("SELECT customer_id, last_name, first_name, DOB, city, state, country, gender FROM CUSTOMERS", conn)
df_customers.head()


,customer_id,last_name,first_name,DOB,city,state,country,gender
0,1,Todd,Kasen,07/30/98,Rock Hill,South Carolina,united_states,m
1,2,Garza,Ensley,04/12/89,singapore,None,singapore,f
2,3,Carey,Lillian,09/12/97,Auburn,Alabama,united_states,f
3,4,Christensen,Beau,01/28/99,Hempstead,New York,united_states,m
4,5,Gibson,Ernesto,03/23/98,singapore,None,singapore,m


### QUESTION 2:

**Extract the relevant data from the CSV file**

For each ```customer_id``` determine if a customer has stopped their subscription or not and save it in a dictionary or another data container.

In [12]:
df_streams = pd.read_csv(DATA_DIR + '/aavail-streams.csv')
df_streams.head()

,customer_id,stream_id,date,subscription_stopped
0,1,1356,2018-12-01,0
1,1,1540,2018-12-04,0
2,1,1395,2018-12-11,0
3,1,1255,2018-12-22,0
4,1,1697,2018-12-23,0


In [13]:
## YOUR CODE HERE
cust_status = df_streams.groupby(['customer_id'], sort=False)['subscription_stopped'].max().to_dict()

## PART 2: Checks for quality assurance

Sometimes it is known in advance which types of data integrity issues to expect, but other times it is during the Exploratory Data Analysis (EDA) process that these issues are identified.  After extracting data it is important to include checks for quality assurance even on the first pass through the AI workflow.  Here you will combine the data into a single structure and provide a couple checks for quality assurance.

### QUESTION 3: 

**Implement checks for quality assurance**

1. Remove any repeat customers based on ```customer_id```
2. Remove stream data that do not have an associated ```stream_id```
3. Check for missing values

### 1. Remove repeat customers based on ```customer_id```

In [14]:
num_dupes = df_customers.duplicated(['customer_id']).sum()
print(f'...Removing {num_dupes} duplicate customer records')
df_customers = df_customers[~df_customers.duplicated(['customer_id'])]

...Removing 7 duplicate customer records


### 2. Remove stream data that do not have an associated ```stream_id```

In [15]:
num_missing_stream_id = df_streams['stream_id'].isna().sum()
print(f'...Removing {num_missing_stream_id} streams with missing IDs')
df_streams = df_streams[~df_streams['stream_id'].isna()]

df_streams.head()

...Removing 0 streams with missing IDs


,customer_id,stream_id,date,subscription_stopped
0,1,1356,2018-12-01,0
1,1,1540,2018-12-04,0
2,1,1395,2018-12-11,0
3,1,1255,2018-12-22,0
4,1,1697,2018-12-23,0


### 3. Check for missing values

In [16]:
#state is missing for country outside US
df_customers.drop(['state'], axis=1).isnull().sum().sum()

0

### QUESTION 4: 

**combine the data into a single data structure**

For this example, the two most convenient structures for this task are Pandas dataframes and NumPy arrays.  At a minimum ensure that your structure accommodates the following.

1. A column for `customer_id`
2. A column for `country`
3. A column for ```age``` that is created from ```DOB```
4. A column ```customer_name``` that is created from ```first_name``` and ```last_name```
5. A column to indicate churn called ```is_subscriber```
7. A column that indicates ```subscriber_type``` that comes from ```invoice_item```
6. A column to indicate the total ```num_streams```

#### Resources

* [Python's datetime library](https://docs.python.org/3/library/datetime.html)
* [NumPy's datetime data type](https://docs.scipy.org/doc/numpy/reference/arrays.datetime.html)


In [17]:
## YOUR CODE HERE
from datetime import date, datetime

def calculate_age(dob):

    dob = datetime.strptime(dob, "%m/%d/%y").date()
    today = date.today()
    
    min_csr_age = 10
    age_ceiling = today.year - min_csr_age
    
    if dob.year > age_ceiling:
        dob = dob.replace(year=dob.year-100)
        
    return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

In [18]:
df_customers['age'] = df_customers['DOB'].apply(calculate_age)

df_customers.head()

,customer_id,last_name,first_name,DOB,city,state,country,gender,age
0,1,Todd,Kasen,07/30/98,Rock Hill,South Carolina,united_states,m,21
1,2,Garza,Ensley,04/12/89,singapore,None,singapore,f,30
2,3,Carey,Lillian,09/12/97,Auburn,Alabama,united_states,f,22
3,4,Christensen,Beau,01/28/99,Hempstead,New York,united_states,m,21
4,5,Gibson,Ernesto,03/23/98,singapore,None,singapore,m,21


In [19]:
print(df_customers['age'].max())
print(df_customers['age'].min())


56
13


In [20]:
df_customers["customer_name"] = df_customers["first_name"] + " " + df_customers["last_name"]
df_customers.head()

,customer_id,last_name,first_name,DOB,city,state,country,gender,age,customer_name
0,1,Todd,Kasen,07/30/98,Rock Hill,South Carolina,united_states,m,21,Kasen Todd
1,2,Garza,Ensley,04/12/89,singapore,None,singapore,f,30,Ensley Garza
2,3,Carey,Lillian,09/12/97,Auburn,Alabama,united_states,f,22,Lillian Carey
3,4,Christensen,Beau,01/28/99,Hempstead,New York,united_states,m,21,Beau Christensen
4,5,Gibson,Ernesto,03/23/98,singapore,None,singapore,m,21,Ernesto Gibson


In [21]:
df_customers["is_subscriber"] = df_customers['customer_id'].map(cust_status)
df_customers.head()

,customer_id,last_name,first_name,DOB,city,state,country,gender,age,customer_name,is_subscriber
0,1,Todd,Kasen,07/30/98,Rock Hill,South Carolina,united_states,m,21,Kasen Todd,0
1,2,Garza,Ensley,04/12/89,singapore,None,singapore,f,30,Ensley Garza,1
2,3,Carey,Lillian,09/12/97,Auburn,Alabama,united_states,f,22,Lillian Carey,0
3,4,Christensen,Beau,01/28/99,Hempstead,New York,united_states,m,21,Beau Christensen,0
4,5,Gibson,Ernesto,03/23/98,singapore,None,singapore,m,21,Ernesto Gibson,0


In [22]:
# df_customer_types = pd.DataFrame(df_invoices[["customer_id","invoice_item_id"]].invoice_item_id.replace(customer_types))
df_subscriber_types = df_invoices.loc[:,["customer_id", "invoice_item_id"]]
df_subscriber_types["subscriber_type"] = df_subscriber_types.invoice_item_id.replace(customer_types)
df_subscriber_types = df_subscriber_types[["customer_id", "subscriber_type"]]

df_customers = pd.merge(df_customers, df_subscriber_types, on='customer_id')
df_customers.head()

,customer_id,last_name,first_name,DOB,city,state,country,gender,age,customer_name,is_subscriber,subscriber_type
0,1,Todd,Kasen,07/30/98,Rock Hill,South Carolina,united_states,m,21,Kasen Todd,0,aavail_unlimited
1,2,Garza,Ensley,04/12/89,singapore,None,singapore,f,30,Ensley Garza,1,aavail_unlimited
2,3,Carey,Lillian,09/12/97,Auburn,Alabama,united_states,f,22,Lillian Carey,0,aavail_premium
3,4,Christensen,Beau,01/28/99,Hempstead,New York,united_states,m,21,Beau Christensen,0,aavail_unlimited
4,5,Gibson,Ernesto,03/23/98,singapore,None,singapore,m,21,Ernesto Gibson,0,aavail_basic


In [23]:
# pd.merge(df_customers, df_streams.groupby(['customer_id'], sort=False).count().reset_index(), on='customer_id')
df_customers = pd.merge(df_customers,df_streams.groupby(['customer_id'], sort=False)['stream_id'].count().reset_index().rename(columns={'stream_id':'num_streams'}), on='customer_id')
df_customers.head()

,customer_id,last_name,first_name,DOB,city,state,country,gender,age,customer_name,is_subscriber,subscriber_type,num_streams
0,1,Todd,Kasen,07/30/98,Rock Hill,South Carolina,united_states,m,21,Kasen Todd,0,aavail_unlimited,21
1,2,Garza,Ensley,04/12/89,singapore,None,singapore,f,30,Ensley Garza,1,aavail_unlimited,16
2,3,Carey,Lillian,09/12/97,Auburn,Alabama,united_states,f,22,Lillian Carey,0,aavail_premium,25
3,4,Christensen,Beau,01/28/99,Hempstead,New York,united_states,m,21,Beau Christensen,0,aavail_unlimited,18
4,5,Gibson,Ernesto,03/23/98,singapore,None,singapore,m,21,Ernesto Gibson,0,aavail_basic,21


In [24]:
df_customers.to_csv(os.path.join(DATA_DIR, "processed_data.csv"))

## PART 3: Automating the process

To ensure that you code can be used to automate this process.  First you will save you dataframe or numpy array as a CSV file.  

### QUESTION 5:

**Take the initial steps towards automation**

1. Save your cleaned, combined data as a CSV file.
2. From the code above create a function or class that performs all of the steps given a database file and a streams CSV file.
3. Run the function in batches and write a check to ensure you got the same result that you did in the code above.

There will be some logic involved to ensure that you do not write the same data twice to the target CSV file.

Shown below is some code that will split your streams file into two batches. 

In [25]:
# body = client_bfb337e9d3b24554ad9a816fe610a744.get_object(Bucket='aienterpriseworkflowmodule1casest-donotdelete-pr-qwt6qhnjsctfni',Key='aavail-streams.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
# if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

## code to split the streams csv into batches
DATA_DIR = "/home/dsxuser/data"
df_all = pd.read_csv(os.path.join(DATA_DIR, 'aavail-streams.csv'))

half = int(round(df_all.shape[0] * 0.5))
df_part1 = df_all[:half]
df_part2 = df_all[half:]

df_part1.size
df_part1.to_csv(os.path.join(DATA_DIR, 'aavail-streams-1.csv'), index=False)
df_part2.to_csv(os.path.join(DATA_DIR, 'aavail-streams-2.csv'), index=False)

In [26]:
os.getcwd()
print(os.listdir("/home/dsxuser/data"))

['aavail-streams-2.csv', 'processed_data.csv', 'aavail-customers.db', 'aavail-streams-1.csv', 'aavail-streams.csv']


In [27]:
%%writefile aavail-data-ingestor.py
#!/usr/bin/env python

import os
import pandas as pd
import numpy as np
import sqlite3
import sys
import getopt
from datetime import date, datetime

DATA_DIR = '/home/dsxuser/data'

def connect_db(file_path):
    try:
        conn = sqlite3.connect(file_path)
        print("...successfully connected to db\n")
    except Error as e:
        print("...unsuccessful connection\n",e)
    
    return(conn)

def calculate_age(dob):

    dob = datetime.strptime(dob, "%m/%d/%y").date()
    today = date.today()
    
    min_csr_age = 10
    age_ceiling = today.year - min_csr_age
    
    if dob.year > age_ceiling:
        dob = dob.replace(year=dob.year-100)
        
    return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

def process_data(db, stream_csv):
    # connect to customer db
    conn = connect_db(db)
    
    # read tables as dataframes
    df_customers = pd.read_sql_query("SELECT customer_id, last_name, first_name, DOB, city, state, country, gender FROM CUSTOMERS", conn)
    df_invoices = pd.read_sql_query("SELECT * FROM INVOICES;", conn)
    df_invoice_items = pd.read_sql_query("SELECT invoice_item_id, invoice_item FROM INVOICE_ITEMS;", conn)
    
    # get subscriber types from invoice items
    subscription_types = df_invoice_items.set_index("invoice_item_id").T.to_dict("index")['invoice_item']
    
    # get streams data
    df_streams = pd.read_csv(stream_csv)

    # get subscriber status from streams
    subscriber_status = df_streams.groupby(['customer_id'], sort=False)['subscription_stopped'].max().to_dict()
    
    # remove duplicate customers
    num_dupes = df_customers.duplicated(['customer_id']).sum()
    print(f'...Removing {num_dupes} duplicate customer records')
    df_customers = df_customers[~df_customers.duplicated(['customer_id'])]
    
    # remove stream data w no stream id
    num_missing_stream_id = df_streams['stream_id'].isna().sum()
    print(f'...Removing {num_missing_stream_id} streams with missing IDs')
    df_streams = df_streams[~df_streams['stream_id'].isna()]
    
    # check for missing values
    print('...Number of missing values: ', df_customers.drop(['state'], axis=1).isnull().sum().sum())           #state is missing for country outside US

    # enrich customer records
    df_customers['age'] = df_customers['DOB'].apply(calculate_age)
    df_customers["customer_name"] = df_customers["first_name"] + " " + df_customers["last_name"]
    df_customers["is_subscriber"] = df_customers['customer_id'].map(subscriber_status)
    
    # subscription type
    df_subscriber_types = df_invoices.loc[:,["customer_id", "invoice_item_id"]]
    df_subscriber_types["subscriber_type"] = df_subscriber_types.invoice_item_id.replace(subscription_types)
    df_subscriber_types = df_subscriber_types[["customer_id", "subscriber_type"]]    
    df_customers = pd.merge(df_customers, df_subscriber_types, on='customer_id')
    
    # number of streams
    df_customers = pd.merge(df_customers,df_streams.groupby(['customer_id'], sort=False)['stream_id'].count().reset_index().rename(columns={'stream_id':'num_streams'}), on='customer_id')

    return df_customers

def update_target(target_file,df_clean,overwrite=False):
    """
    update line by line in case data are large
    """

    if overwrite or not os.path.exists(target_file):
        df_clean.to_csv(target_file,index=False)   
    else:
        df_target = pd.read_csv(target_file)
        df_target.to_csv(target_file, mode='a',index=False)
         
if __name__ == "__main__":
  
    ## collect args
    arg_string = "%s -d db_filepath -s streams_filepath"%sys.argv[0]
    try:
        optlist, args = getopt.getopt(sys.argv[1:],'d:s:')
    except getopt.GetoptError:
        print(getopt.GetoptError)
        raise Exception(arg_string)

    ## handle args
    streams_file = None
    db_file = None
    for o, a in optlist:
        if o == '-d':
            db_file = a
        if o == '-s':
            streams_file = a
    streams_file = os.path.join(DATA_DIR,streams_file)
    db_file = os.path.join(DATA_DIR,db_file)
    target_file = os.path.join(DATA_DIR,"aavail-target.csv")
    
    ## process data
    df_clean = process_data(db_file, streams_file)
    
    ## write
    update_target(target_file,df_clean,overwrite=False)
    print("done")

Writing aavail-data-ingestor.py


In [28]:
db_file = os.path.join(DATA_DIR, 'aavail-customers.db')
streams_csv = os.path.join(DATA_DIR, 'aavail-streams.csv')

process_data(db_file, streams_csv).to_csv(os.path.join(DATA_DIR, "processed_data.csv"))
!wc -l ../data/processed_data.csv

NameError: name 'process_data' is not defined

In [48]:
!python aavail-data-ingestor.py -d aavail-customers.db -s aavail-streams.csv

...successfully connected to db

...Removing 7 duplicate customer records
...Removing 0 streams with missing IDs
...Number of missing values:  0
done


In [32]:
!mv ../data/aavail-target.csv ../data/aavail-target-complete.csv
!wc -l ../data/aavail-target-complete.csv

1001 ../data/aavail-target-complete.csv


In [33]:

!python aavail-data-ingestor.py -d aavail-customers.db -s aavail-streams-1.csv
!wc -l ../data/aavail-target.csv
!python aavail-data-ingestor.py -d aavail-customers.db -s aavail-streams-2.csv
!wc -l ../data/aavail-target.csv

...successfully connected to db

...Removing 7 duplicate customer records
...Removing 0 streams with missing IDs
...Number of missing values:  0
done
498 ../data/aavail-target.csv
...successfully connected to db

...Removing 7 duplicate customer records
...Removing 0 streams with missing IDs
...Number of missing values:  0
done
996 ../data/aavail-target.csv


In [45]:
!python aavail-data-ingestor.py -d aavail-customers.db -s aavail-streams-1.csv
!wc -l ../data/aavail-target.csv

...successfully connected to db

...Removing 7 duplicate customer records
...Removing 0 streams with missing IDs
...Number of missing values:  0
done
498 ../data/aavail-target.csv


In [49]:
!head ../data/aavail-target.csv

customer_id,last_name,first_name,DOB,city,state,country,gender,age,customer_name,is_subscriber,subscriber_type,num_streams
1,Todd,Kasen,07/30/98,Rock Hill,South Carolina,united_states,m,21,Kasen Todd,0,aavail_unlimited,21
2,Garza,Ensley,04/12/89,singapore,,singapore,f,30,Ensley Garza,1,aavail_unlimited,16
3,Carey,Lillian,09/12/97,Auburn,Alabama,united_states,f,22,Lillian Carey,0,aavail_premium,25
4,Christensen,Beau,01/28/99,Hempstead,New York,united_states,m,21,Beau Christensen,0,aavail_unlimited,18
5,Gibson,Ernesto,03/23/98,singapore,,singapore,m,21,Ernesto Gibson,0,aavail_basic,21
6,Murray,Deshawn,09/18/97,Portland,Maine,united_states,m,22,Deshawn Murray,0,aavail_premium,22
7,Tate,Daxton,12/23/70,singapore,,singapore,m,49,Daxton Tate,1,aavail_unlimited,16
8,Small,Tenley,07/21/72,Paterson,New Jersey,united_states,f,47,Tenley Small,0,aavail_basic,21
9,Chase,Kyra,05/19/98,Temple,Texas,united_states,f,21,Kyra Chase,0,aavail_premium,4


In [ ]:
!csvdiff --style=summary ./data/processed_data.csv ../data/aavail-target.csv 

In [35]:
import difflib
csv1 = open(os.path.join(DATA_DIR, "aavail-target-complete.csv"), 'r').readlines()
csv2 = open(os.path.join(DATA_DIR, "aavail-target.csv"), 'r').readlines()
    
for line in csv1:
    print(difflib.unifiedline)

--- 

+++ 

@@ -1,1001 +1,996 @@

 customer_id,last_name,first_name,DOB,city,state,country,gender,age,customer_name,is_subscriber,subscriber_type,num_streams

-1,Todd,Kasen,07/30/98,Rock Hill,South Carolina,united_states,m,21,Kasen Todd,0,aavail_unlimited,21

-2,Garza,Ensley,04/12/89,singapore,,singapore,f,30,Ensley Garza,1,aavail_unlimited,16

-3,Carey,Lillian,09/12/97,Auburn,Alabama,united_states,f,22,Lillian Carey,0,aavail_premium,25

-4,Christensen,Beau,01/28/99,Hempstead,New York,united_states,m,21,Beau Christensen,0,aavail_unlimited,18

-5,Gibson,Ernesto,03/23/98,singapore,,singapore,m,21,Ernesto Gibson,0,aavail_basic,21

-6,Murray,Deshawn,09/18/97,Portland,Maine,united_states,m,22,Deshawn Murray,0,aavail_premium,22

-7,Tate,Daxton,12/23/70,singapore,,singapore,m,49,Daxton Tate,1,aavail_unlimited,16

-8,Small,Tenley,07/21/72,Paterson,New Jersey,united_states,f,47,Tenley Small,0,aavail_basic,21

-9,Chase,Kyra,05/19/98,Temple,Texas,united_states,f,21,Kyra Chase,0,aavail_premium,4



+94,Travis,Tristan,04/04/78,Frisco,Texas,united_states,m,41,Tristan Travis,0.0,aavail_unlimited,18

+95,Melton,Chloe,04/24/81,singapore,,singapore,f,38,Chloe Melton,1.0,aavail_basic,19

+96,Lewis,Dylan,06/05/93,Rochester,New York,united_states,m,26,Dylan Lewis,0.0,aavail_basic,19

+97,Mcconnell,Ellis,10/19/83,singapore,,singapore,m,36,Ellis Mcconnell,1.0,aavail_premium,18

+98,Lara,Nikolai,09/10/64,singapore,,singapore,m,55,Nikolai Lara,1.0,aavail_unlimited,11

+99,Bell,Orion,07/29/94,Mankato,Minnesota,united_states,m,25,Orion Bell,0.0,aavail_premium,16

+100,Roberson,Rylan,12/28/99,singapore,,singapore,m,20,Rylan Roberson,0.0,aavail_basic,22

+101,Reid,Brysen,04/03/98,La Habra,California,united_states,m,21,Brysen Reid,0.0,aavail_unlimited,21

+102,Blanchard,Myra,08/23/71,Asheville,North Carolina,united_states,f,48,Myra Blanchard,0.0,aavail_unlimited,18

+103,Spence,Jordy,01/01/87,Corvallis,Oregon,united_states,m,33,Jordy Spence,0.0,aavail_unlimited,23

+104,Copeland,Alijah,05/15/00,Do

In [42]:
!head ../data/aavail-streams.csv

In [43]:
!head ../data/aavail-streams-1.csv

customer_id,stream_id,date,subscription_stopped
1,1356,2018-12-01,0
1,1540,2018-12-04,0
1,1395,2018-12-11,0
1,1255,2018-12-22,0
1,1697,2018-12-23,0
1,1522,2019-01-04,0
1,1170,2019-01-06,0
1,1314,2019-01-11,0
1,1448,2019-02-07,0


You will need to save your function as a file.  The following cell demonstrates how to do this from within a notebook. 

In [ ]:
%%writefile aavail-data-ingestor.py
#!/usr/bin/env python

import os
import pandas as pd
import numpy as np
import sqlite3

data_dir = os.path.join(".","data")

pass

You will also need to be able to pass the file names to your function without hardcoding them into the script itself.  This is an important step towards automation.  Here are the two libraries commonly used to accomplish this in Python.

* [getopt](https://docs.python.org/3/library/getopt.html)
* [argparse](https://docs.python.org/3/library/argparse.html)

You may run the script you just created from the commandline directly or from within this notebook using:

Run the script once for each batch that you created and then load both the original and batch versions back into the notebook to check that they are the same. 

### QUESTION 6:

**How can you improve the process?**

In paragraph form or using bullets write down some of the ways that you could improve this pipeline.

YOUR ANSWER HERE


